# Overview
------

imports
data setup

interpretation
data distribution
image types (wsi and tma)

Preprocessing
image tileing
image augmentation

Model




In [5]:
!pip install timm

In [6]:
#OBC Ovarian Cancer
#Gregory Sylvester

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import cv2
from tqdm import tqdm
import torchvision.transforms as t
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import timm
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform


In [7]:
df = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv")
df = df[df["is_tma"] == False]
df_np = df.to_numpy()
#for the sake of the project I decided not to work on the WSI (whole slide images) instead focus on TMA (tissue microarray)
#this helps to significantly reduce the time needed to complete the project
df

#below is dataframe

FileNotFoundError: ignored

# Data Exploration
------
Look at the class distribution of the cancer types in the training thumbnail dataset

In [ ]:
# class distibution in histogram
cancers, counts = np.unique(df_np[:,1], return_counts = True)
print(cancers, counts)
plt.bar(cancers, counts)

Looking at the bar graph above we can see for the WSI thumbnails the distibution is not fully balanced so when we split the data we will need to account for this

# Preprocessing, Tuning, & Configuration
-----

In this step the thumbnails need to receive tiles to put in the model. This will involve taking the image transforming it to produce another image:

rotation


In [ ]:
epochs = 10
epoch = epochs
lossRate = .05
seed = 22
torch.manual_seed(seed)
bs = 10
verfSize = .1


def config(epochs_in = epochs, lr_in = lossRate, seed_in = seed, batch_size_in = bs):
    epochs = epochs_in
    lossRate = lr_in
    seed = seed_in
    torch.manual_seed(seed_in)
    bs = batch_size_in

config()

In [ ]:
id_train, id_test, class_train, class_test = 0 , 0,0,0

In [ ]:
def split_data(seed_in, print_values=False):
    id_train, id_verf,class_train,class_verf = train_test_split(df_np[:,0],df_np[:,1], test_size = verfSize, random_state = seed_in)
    if print_values == True:
        print(f"id_train:\n{id_train[:3]}\nclass_train:\n{class_train[:3]}\nid_verf:\n{id_verf[:3]}\nclass_verf:\n{class_verf[:3]}")
    return id_train, id_verf,class_train,class_verf

def filepathThumbs(files):
    pathfiles = []
    print(files)
    for i in range(files.shape[0]):
        pathfiles.append(f"/kaggle/input/UBC-OCEAN/train_thumbnails/{files[i-1]}_thumbnail.png")
    return pathfiles
# note potiential change could be chaging the distibution to be evenly or random instead of default

id_train, id_verf,class_train,class_verf = split_data(22,True)

In [ ]:
def tileSelection(imgs, tile_type = 'r', tile_size = 260):
    files_list = filepathThumbs(imgs)
    num_files = len(files_list)
    out = np.zeros((num_files, tile_size, tile_size, 3), dtype=np.uint8)

    with tqdm(total=num_files, desc="Tiling images", unit="image") as pbar:

        for i, filex in enumerate(files_list):
            img = cv2.imread(filex)

            #here add more if statements for different tiling methods later on as a possible improvement to the model
            if tile_type == 'r':
                rand_row = random.randint(0, img.shape[0] - tile_size)
                rand_col = random.randint(0, img.shape[1] - tile_size)

                tile = img[rand_row:rand_row + tile_size, rand_col:rand_col + tile_size, :]

                out[i, :, :, :] = tile

            pbar.update(1)

    return out


#rand_set_thumb = tileSelection(id_train)
#here chagning the tile size might help to improve the results of the model built

In [ ]:
train_tiles = tileSelection(id_train)
verf_tiles = tileSelection(id_verf)

In [ ]:
train_tiles.shape

In [ ]:
class dataset(Dataset):
    def __init__(self, tiled_images, labels, transform=None):
        self.tiled_images = tiled_images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.tiled_images)

    def __getitem__(self, idx):
        tiled_image = self.tiled_images[idx]
        label = self.labels[idx]
        if self.transform:
            tiled_image = self.transform(tiled_image)

        return tiled_image, label

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
label_encoder = LabelEncoder()

class_train = torch.tensor(label_encoder.fit_transform(class_train), dtype=torch.float32)
class_verf = torch.tensor(label_encoder.fit_transform(class_verf), dtype=torch.float32)



In [ ]:

train_dataset = dataset(train_tiles, class_train,transform)
val_dataset = dataset(verf_tiles, class_verf,transform)




In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=bs, shuffle=False)

# Model Setup
--------
Below is where the model is created and setup
#insert more about timm model

In [ ]:
class cancerClassifier(nn.Module):
    def __init__(self):
        super(cancerClassifier,self).__init__()
        self.num_classes = 6
        self.model = timm.create_model('tf_efficientnet_b2', in_chans=3,checkpoint_path='/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b2/1/tf_efficientnet_b2_aa-60c94f97.pth', pretrained = False)
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(in_features, 6)


    def forward(self, X):
        return self.model(X)



In [ ]:
model = cancerClassifier()
print(model)

# Model Training

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lossRate)


In [ ]:
plt.ion()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
t_hist = []
v_hist = []
total_correct_train = 0
total_samples_train = 0
model.to(device)



for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_correct_train = 0
    total_samples_train = 0
    print(f"Epoch {epoch + 1}\n===============")

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{epochs}', unit='batch') as pbar:
        for inputs, labels in train_loader:

            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.to(torch.long)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            pbar.update(1)

            _, predictions_train = torch.max(outputs, 1)
            total_correct_train += (predictions_train == labels).sum().item()
            total_samples_train += labels.size(0)

        average_loss = total_loss / len(train_loader)

        average_loss_train = total_loss / len(train_loader)
        accuracy_train = total_correct_train / total_samples_train
        t_hist.append(accuracy_train)

        print(f"training accuracy: {accuracy_train}")


    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0

        for inputs, labels in val_loader:

            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.to(torch.long)
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

        accuracy = total_correct / total_samples

        v_hist.append(accuracy)

    print(f"validation accuracy : {accuracy}")
